In [1]:
import pandas as pd
import numpy as np

In [2]:
orders = pd.read_csv("olist_orders_dataset.csv")
order_items = pd.read_csv("olist_order_items_dataset.csv")
payments = pd.read_csv("olist_order_payments_dataset.csv")
reviews = pd.read_csv("olist_order_reviews_dataset.csv")
customers = pd.read_csv("olist_customers_dataset.csv")
products = pd.read_csv("olist_products_dataset.csv")
sellers = pd.read_csv("olist_sellers_dataset.csv")
category_map = pd.read_csv("product_category_name_translation.csv")
# importing all the datasets

In [3]:
orders.shape

(99441, 8)

In [4]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [5]:
orders.isna().sum()
# we can find null values in 3 columns ("order_approved_at",'order_delivered_carrier_date','order_delivered_customer_date')

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

#Understanding the Data

In [6]:
orders['order_status'].value_counts(normalize=True)*100
# we can see the percentage of orders in different stages
# we can see a total of ~1.2% of the orders are cancelled /unavailable which shows the origin of losses

order_status
delivered      97.020344
shipped         1.113223
canceled        0.628513
unavailable     0.612423
invoiced        0.315765
processing      0.302692
created         0.005028
approved        0.002011
Name: proportion, dtype: float64

In [7]:
date_cols = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_customer_date',
    'order_estimated_delivery_date'
]

for col in date_cols:
    orders[col] = pd.to_datetime(orders[col], errors='coerce')
    # converting the object fields to date time.

In [8]:
orders['delivery_delay_days']=(orders['order_delivered_customer_date']-orders['order_estimated_delivery_date']).dt.days

In [9]:
orders['delivery_delay_days'].describe()
# we can see an average of 12 days to deliver a package and even went up to 188days at the Max

count    96476.000000
mean       -11.876881
std         10.183854
min       -147.000000
25%        -17.000000
50%        -12.000000
75%         -7.000000
max        188.000000
Name: delivery_delay_days, dtype: float64

In [10]:
# when delay in delivery increases the reviews gradually go Down

In [11]:
orders_reviews = orders.merge(
    reviews[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

orders_reviews.groupby('review_score')['delivery_delay_days'].mean()

review_score
1.0    -4.060580
2.0    -8.634818
3.0   -10.774052
4.0   -12.380840
5.0   -13.388153
Name: delivery_delay_days, dtype: float64

In [12]:
# As per result it is completely opposite ,so lets check for the delayed data

In [13]:
late_orders = orders_reviews[orders_reviews['delivery_delay_days'] > 0]

late_orders.groupby('review_score')['delivery_delay_days'].mean()


review_score
1.0    12.360337
2.0    10.233813
3.0     9.058739
4.0     8.414110
5.0     6.989623
Name: delivery_delay_days, dtype: float64

In [14]:
# it is confirmed that when the delay goes up the reviews goes down

In [15]:
customer_order_counts=orders.groupby('customer_id')['order_id'].count()
customer_order_counts.value_counts().head(10)

order_id
1    99441
Name: count, dtype: int64

In [16]:
customers.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [17]:
orders_customers = orders.merge(
    customers[['customer_id', 'customer_unique_id']],on='customer_id',how='left')


In [18]:
customer_order_counts = orders_customers.groupby('customer_unique_id')['order_id'].count()
customer_order_counts.value_counts().head(10)


order_id
1     93099
2      2745
3       203
4        30
5         8
6         6
7         3
9         1
17        1
Name: count, dtype: int64

In [19]:
order_items[['seller_id','product_id']].nunique()
# we can see that there are 3095 sellers and 32951 unique products

seller_id      3095
product_id    32951
dtype: int64

# Feature Engineering

In [ ]:
orders['delivery_delay_days']=(orders['order_delivered_customer_date']-orders['order_estimated_delivery_date']).dt.days
# we are finding the no of delayed delivery days

In [ ]:
orders['is_late_delivery']=orders['delivery_delay_days']>0
# creating a new categorical cloumn that tells if it is a delayed delivery or not

In [ ]:
orders['is_late_delivery'].value_counts()
# we can see a total of 6535 delayed deliveries

is_late_delivery
False    92906
True      6535
Name: count, dtype: int64

In [ ]:
orders['is_late_delivery'].mean()*100
# 6.57% of deliveries are delayed

np.float64(6.571736004263835)

In [ ]:
order_revenue = payments.groupby('order_id')['payment_value'].sum().reset_index()
# we are segmenting the revenue per order
orders = orders.merge(order_revenue, on='order_id', how='left')
# merging the payment_value of each order from the order_revenue dataframe

In [40]:
order_revenue.head()
# Dataframe that contains the payment_value for each order

,order_id,payment_value
0,00010242fe8c5a6d1ba2dd792cb16214,72.19
1,00018f77f2f0320c557190d7a144bdd3,259.83
2,000229ec398224ef6ca0657da4fc703e,216.87
3,00024acbcdf0a6daa1e931b038114c75,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,218.04


In [42]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,is_late_delivery,payment_value,churn_flag,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,-8.0,False,38.71,True,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,-6.0,False,141.46,False,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,-18.0,False,179.12,False,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,-13.0,False,72.20,True,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,-10.0,False,28.62,True,5.0


In [30]:
orders['payment_value'].describe()

count    99440.000000
mean       160.990267
std        221.951257
min          0.000000
25%         62.010000
50%        105.290000
75%        176.970000
max      13664.080000
Name: payment_value, dtype: float64

In [ ]:
latest_date = orders['order_purchase_timestamp'].max()

last_order = orders.groupby('customer_id')['order_purchase_timestamp'].max()

customer_features = pd.DataFrame({
    'last_order_date': last_order
})

customer_features['days_since_last_order'] = (
    latest_date - customer_features['last_order_date']
).dt.days

customer_features['churn_flag'] = customer_features['days_since_last_order'] > 90
# we are assuming the a customer has churned if the difference between the current date and 
# last order is more than 90 days.

In [ ]:
customer_features['churn_flag'].mean()*100
# churn Percentage.

np.float64(90.15597188282499)

In [ ]:
orders = orders.merge(
    customer_features[['churn_flag']],
    left_on='customer_id',
    right_index=True,
    how='left'
)
# we are merging the churn_flag with the orders table using the customer_id.

In [ ]:
orders = orders.merge(
    reviews[['order_id', 'review_score']],
    on='order_id',
    how='left'
)
# we are mergig the review_score in the order table to identify the score per order.

In [35]:
orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,delivery_delay_days,is_late_delivery,payment_value,churn_flag,review_score
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,-8.0,False,38.71,True,4.0
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,-6.0,False,141.46,False,4.0
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,-18.0,False,179.12,False,5.0
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,-13.0,False,72.20,True,5.0
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,-10.0,False,28.62,True,5.0


In [37]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99992 entries, 0 to 99991
Data columns (total 13 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99992 non-null  object        
 1   customer_id                    99992 non-null  object        
 2   order_status                   99992 non-null  object        
 3   order_purchase_timestamp       99992 non-null  datetime64[ns]
 4   order_approved_at              99831 non-null  datetime64[ns]
 5   order_delivered_carrier_date   98199 non-null  object        
 6   order_delivered_customer_date  97005 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99992 non-null  datetime64[ns]
 8   delivery_delay_days            97005 non-null  float64       
 9   is_late_delivery               99992 non-null  bool          
 10  payment_value                  99991 non-null  float64       
 11  churn_flag     